In [ ]:
from collections import namedtuple
import pathlib
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (
    measurement as om,
    net_quantity as onq,
    project_loader as opl,
)

In [ ]:
import toolz.curried as toolz

In [ ]:
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
import UnitsNet
from System import Array, Int32

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
@toolz.curry
def to_project_unit(unit, nq):
    return nq.ToUnit(unit)

@toolz.curry
def net_quantity_to_tuple(nq):
    return nq.Value, nq.Unit

In [ ]:
def get_project_wells(proj):
    return proj.Wells.Items

def collect_well_names(wells):
    return {w.Name: w for w in wells}

project_wells = toolz.pipe(
    projects,
    toolz.valmap(get_project_wells),
    toolz.valmap(collect_well_names),
)
project_wells

In [ ]:
@toolz.curry
def summarize_well_measurements(well):
    return {'KB above ground': str(well.KellyBushingHeightAboveGroundLevel),
            'GL above sea level': str(well.GroundLevelElevationAboveSeaLevel,),}

well_measurements = toolz.pipe(project_wells,
                               toolz.valmap(toolz.valmap(summarize_well_measurements)))
well_measurements

In [ ]:
frames = {
    WellReferenceFrameXy.AbsoluteStatePlane: 'Plane',
    WellReferenceFrameXy.Project: 'Project',
    WellReferenceFrameXy.WellHead: 'Well',
}

datums = {
    DepthDatum.GroundLevel: 'Ground',
    DepthDatum.KellyBushing: 'Kelly',
    DepthDatum.SeaLevel: 'Sea',
}

In [ ]:
frames

In [ ]:
selected_field_name = 'montney'

In [ ]:
# selected_well_name = 'Hori_01'
# selected_well_name = 'Hori_02'
selected_well_name = 'Hori_03'

In [ ]:
# # Hori_01
# selected_frame = WellReferenceFrameXy.AbsoluteStatePlane
# selected_datum = DepthDatum.KellyBushing

# # Hori_02
# selected_frame = WellReferenceFrameXy.WellHead
# selected_datum = DepthDatum.GroundLevel

# Hori_03
selected_frame = WellReferenceFrameXy.Project
selected_datum = DepthDatum.SeaLevel

In [ ]:
selected_well = toolz.get_in([selected_field_name, selected_well_name], project_wells)
selected_well

In [ ]:
selected_well_md_kb_values = selected_well.Trajectory.GetMdKbArray()
len(selected_well_md_kb_values)

In [ ]:
trajectory_bounds_measurements = toolz.pipe(
    selected_well_md_kb_values,
    toolz.get([0, -1]),
)

toolz.pipe(trajectory_bounds_measurements,
           toolz.map(str),
           list)

In [ ]:
trajectory_bounds = toolz.pipe(trajectory_bounds_measurements,
                               toolz.map(lambda v: v.Value),
                               list)
trajectory_bounds

In [ ]:
def sample(bounds):
    return random.uniform(*bounds)

sample_size = 4
md_kb_at = toolz.pipe(range(sample_size),
                      toolz.map(lambda _: sample(trajectory_bounds)),
                      sorted,
                      list)
md_kb_at

In [ ]:
def make_net_feet(v):
    return onq.net_length_from_ft(v)

def make_net_meters(v):
    return onq.net_length_from_m(v)

@toolz.curry
def make_net_quantity(make_quantity_func, value):
    return make_quantity_func(value)


# net_quantity_func = make_net_feet
net_quantity_func = make_net_meters

# sample_md_kb_measurements = toolz.pipe(
#     md_kb_at,
#     toolz.map(make_net_quantity(net_quantity_func)),
#     list
# )
sample_md_kb_measurements = toolz.pipe(
    # [467.3, 814.8, 2595., 3062.],  # Hori_02
    # [1026., 1889., 3968., 4096.],  # Hori_02
    [358.8, 1018., 3668., 4439.],  # Hori_03
    toolz.map(make_net_quantity(net_quantity_func)),
    list,
)

[str(m) for m in sample_md_kb_measurements]

In [ ]:
sample_values = Array[UnitsNet.Length](sample_md_kb_measurements)
for i in range(len(sample_values)):
    print(str(sample_values[i]))

In [ ]:
SimplePoint = namedtuple('SimplePoint', ['x', 'y', 'z'])

In [ ]:
raw_points = selected_well.GetLocationsForMdKbValues(sample_values, 
                                                     selected_frame,
                                                     selected_datum)
points = toolz.pipe(
    raw_points,
    toolz.map(lambda p: SimplePoint(p.X.ToUnit(project_units[selected_field_name].LengthUnit),
                                    p.Y.ToUnit(project_units[selected_field_name].LengthUnit),
                                    p.Depth.ToUnit(project_units[selected_field_name].LengthUnit))),
    list,
)

In [ ]:
for point in points:
    print(f'SubsurfacePoint({str(point.x)}, {str(point.y)}, {str(point.z)})')

In [ ]:
def abbreviation(u):
    return UnitsNet.Length.GetAbbreviation(u)
    
def format_length(v):
    return f'{v.Value:#.4g} {abbreviation(v.Unit)}'
    
@toolz.curry
def summarize_point(e):
    sample_measurement, point = e
    return {
        'field': selected_field_name,
        'well': selected_well_name,
        'mb_kb': f'{format_length(sample_measurement)}',
        'frame': frames[selected_frame],
        'datum': datums[selected_datum],
        'x': f'{format_length(point.x)}',
        'y': f'{format_length(point.y)}',
        'z': f'{format_length(point.z)}',
    }


point_summary = toolz.pipe(
    zip(sample_md_kb_measurements, points),
    toolz.map(summarize_point),
    list,
)
point_summary

In [ ]:
pd.DataFrame(data=point_summary)

In [ ]:
whl_data = []
def combine_data(whl_loc):
    return str(whl_loc.Value) + ' ' + abbreviation(whl_loc.Unit)

for proj_name in project_wells:
    for wellname, well in project_wells[proj_name].items():
        whl = well.WellHeadLocation
        whl_data.append((proj_name,
                         wellname,
                         combine_data(whl[0]),
                         combine_data(whl[1]),
                         combine_data(whl[2])))

whl_df = pd.DataFrame(whl_data,
                      columns=['Project',
                               'Well',
                               'Easting',
                               'Northing',
                               'Depth'])
whl_df.head(whl_df.shape[0])
